In [1]:
def quadrantdaywise():
    import random
    import requests
    import urllib3
    urllib3.disable_warnings()
    import pandas as pd
    import numpy as np
    import re
    import string
    import time
    import win32com.client
    from datetime import date, timedelta
    import os
    from os import listdir
    from os.path import join
    import shutil
    import datetime
    import matplotlib.pyplot as plt
    import seaborn as sns
    import warnings
    warnings.filterwarnings("ignore")
    from sklearn.model_selection import train_test_split
    from sklearn.impute import SimpleImputer
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
    from sklearn.ensemble import RandomForestClassifier
    from xgboost import XGBClassifier

    import ssl

    -
    try:
        now = datetime.datetime.now()
        start_date = "{}".format((now-timedelta(hours=24, seconds=0, minutes=0)).strftime("%Y-%m-%d"))
#         start_date = f'2021-09-0{i}'
        end_date = start_date
        data =  {
            "type": "per_day",
            "survey_id":[85],
            "limit": 1
        }
        
        proxies = ['http://192.168.5.37:8080',
                   'http://192.168.5.38:8080',
                   'http://192.168.5.39:8080',
                   'http://192.168.5.40:8080',
                   'http://192.168.5.41:8080',
                   'http://192.168.5.42:8080',
                   'http://192.168.5.43:8080',
                   'http://192.168.5.44:8080',
                   'http://192.168.5.45:8080',
                   'http://192.168.5.46:8080']

        import random
        a = random.randrange(len(proxies))
        proxy = {"http" : f"{proxies[a]}"}
        
        res = requests.post('https://npci.surveycxm.com:3000/survey/get-merge-xlsx-file',json =data ,verify = False, proxies=proxy)

        url = res.json()['data'][0]['file_url']
        df = pd.read_excel(url)

        def dotask(df):
            def clean_text(text ): 
                delete_dict = {sp_character: '' for sp_character in string.punctuation} 
                delete_dict[' '] = ' ' 
                table = str.maketrans(delete_dict)
                text1 = text.translate(table)
                #print('cleaned:'+text1)
                textArr= text1.split()
                text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
                return text2.lower()


            df['Remarks'] =df['Remarks'].astype(str)
            df['Cleaned_Remarks'] = ''
            df['Cleaned_Remarks'] =df['Remarks'].apply(clean_text)
            df['Categories'] = ''
            df['Sub Categories'] = ''


            Happy_Customers     = ['very good ','simple transfer ','easy ','good ','efficient way ','fast app ','more accessible ','better processing ',
                                   'very effective ','fast ','user friendly ','best upi ','quite efficient ','so simple ','very safe ','indian ',
                                   'very quickly ','best ','nice ','very satisfied ','happy ','love ','just wonderful ','super ','awesome ','best performance ',
                                   'excellent ','nice app ','for pay bill easily ','nice service ','best customer experience ','satisfied ','better app ',
                                   'very nice ','excellent ','fabulous ','awsome ','awesome ','cool ', 'very good',' simple transfer',' easy',' good',' efficient way',' fast app',' more accessible',' better processing',
                                   ' very effective',' fast',' user friendly',' best upi',' quite efficient',' so simple',' very safe',' indian',
                                   ' very quickly',' best',' nice',' very satisfied',' happy',' love',' just wonderful',' super',' awesome',' best performance',
                                   ' excellent',' nice app',' for pay bill easily',' nice service',' best customer experience',' satisfied',' better app',
                                   ' very nice',' excellent',' fabulous',' awsome',' awesome',' cool']
            Transaction                  = ['transaction ','transition ','transection ',' transaction',' transition',' transection']
            Recharge                     = ['recharge ','recharging ',' recharge',' recharging']
            Comparing                    = ['works faster than ','compare ','way better than ','better than ','googlepay ','google pay ','paytm ','phonepay ', 'phone pay ','mi ','other upi ',
                                           'mobikwik ','gpay ','phonepes ','paypal ',' works faster than',' compare',' way better than',' better than',' googlepay',
                                            ' google pay',' paytm',' phonepay',' phone pay',' mi',' other upi',' mobikwik',' gpay',' phonepes',' paypal']
            Interface                    = ['interface ','inter face ',' interface',' inter face']
            Server                       = ['very slow ','speed ','server ','network issue ','low speed','servsr ',' very slow',' speed',' server',' network issue',' low speed',' servsr']
            Complaints                   = ['complaint ','fraud ','upi pin ','complaints ','unblock ','thing went wrong ','disabled ','complaint ',
                                            'fraud ','upi pin ','complaints ','unblock ','thing went wrong ','disabled ']
            Isses_Not_Solved             =  ['issue ','issues ','not resolved ',' issue',' issues',' not resolved']
            Offers_And_Reward            =  ['reward ','discount ','offer ','rewards ','cashback ','scratch card ','cashbach ','vouchers ','cashbacks ','cash back ','coupons ',
                                              'redeem card ',' reward',' discount',' offer',' rewards',' cashback',' scratch card',' cashbach',' vouchers',' cashbacks',' cash back',' coupons',
                                            ' redeem card']
            Customer_Service             =  ['customer care ','customer service ','customer support ','toll free ','customers care ', ' customer care',' customer service',' customer support',
                                           ' toll free',' customers care']
            Suggestion_For_Improvement   = ['add ','improve ','improvement ','improvements ','start new ','more options ','more option ',
                                           'Make better ','feedback given by ','more update ','improvment ','more facilities ','booking features ',' add',' improve',' improvement',
                                            ' improvements',' start new',' more options',' more option', ' Make better',' feedback given by',' more update',' improvment',
                                           ' more facilities',' booking features']
            QR_Code                      = [' qr','qr ']
            Security                     = ['security ','security issues ',' security',' security issues']
            Facing_Problem_In_App        = [' can not see',' cannt see',' cant see', 'can not see ','cannt see ','cant see ']
            Related_to_Bugs              = ['bugs ','bug ','fix it ',' bugs',' bug',' fix it']
            Not_Happy_With_App           = ['hopeless ','pathetic ','hate ','bad app ','not good ','app not working ',' hopeless',' pathetic',' hate',' bad app',' not good',' app not working']
            Not_Able_To_Login            =['login ','log in ','sign in ','signin ','sim ','late ','verfication ',' login',' log in',' sign in',' signin',' sim',' late',' verfication']
            IPO                          = ['ipo ',' ipo']


            for i in range(0, len(df.Cleaned_Remarks)):
                if any(word in df.Cleaned_Remarks[i] for word in Happy_Customers):
                    df['Categories'][i] =  'Happy Customers'

                elif any(word in df.Cleaned_Remarks[i] for word in Transaction):
                    df['Categories'][i] = 'Transaction'

                elif any(word in df.Cleaned_Remarks[i] for word in Recharge):
                    df['Categories'][i] = 'Recharge'

                elif any(word in df.Cleaned_Remarks[i] for word in Comparing):
                    df['Categories'][i] = 'Comparing'

                elif any(word in df.Cleaned_Remarks[i] for word in Interface):
                    df['Categories'][i] = 'Interface'

                elif any(word in df.Cleaned_Remarks[i] for word in Server):
                    df['Categories'][i] = 'Server'

                elif any(word in df.Cleaned_Remarks[i] for word in Complaints):
                    df['Categories'][i] = 'Complaints'

                elif any(word in df.Cleaned_Remarks[i] for word in Isses_Not_Solved):
                    df['Categories'][i] = 'Isses_Not_Solved'

                elif any(word in df.Cleaned_Remarks[i] for word in Offers_And_Reward):
                    df['Categories'][i] = 'Offers_And_Reward'

                elif any(word in df.Cleaned_Remarks[i] for word in Customer_Service):
                    df['Categories'][i] = 'Customer_Service'

                elif any(word in df.Cleaned_Remarks[i] for word in Suggestion_For_Improvement):
                    df['Categories'][i] = 'Suggestion For Improvement'

                elif any(word in df.Cleaned_Remarks[i] for word in QR_Code):
                    df['Categories'][i] = 'QR Code'

                elif any(word in df.Cleaned_Remarks[i] for word in Security):
                    df['Categories'][i] = 'Security'

                elif any(word in df.Cleaned_Remarks[i] for word in Facing_Problem_In_App):
                    df['Categories'][i] = 'Facing Problem In App'

                elif any(word in df.Cleaned_Remarks[i] for word in Related_to_Bugs):
                    df['Categories'][i] = 'Related to Bugs'

                elif any(word in df.Cleaned_Remarks[i] for word in Not_Happy_With_App):
                    df['Categories'][i] = 'Not Happy With App'


                elif any(word in df.Cleaned_Remarks[i] for word in Not_Able_To_Login):
                    df['Categories'][i] = 'Not Able To Login'

                elif any(word in df.Cleaned_Remarks[i] for word in IPO):
                    df['Categories'][i] = 'IPO'


            Transaction = df[df.Categories=='Transaction']
            Transaction =Transaction.reset_index()
            Transaction = Transaction.drop(['index'],axis =1)

            Not_able_to_do_transcation = ['mobile number has been changed','transfer','payement not','payment not','processed',
                                          'can not debited','cant debited','can not debited']
            wrong_transaction = ['wrong transaction','mistakly wrong transaction']
            money_dedected = ['detected','deduct','deducted']
            transaction_limit = ['limit','trasaction limit']
            payment_failure = ['failure','fails','payment failure','payment fails']
            not_refund = ['not refund','refund','refunding','refunds']
            error = ['error','errors']
            other_issue = ['issue','other issue','issues']
            pending = ['pending']
            rejection_in_payment = ['rejection']

            for i in range(0, len(Transaction.Cleaned_Remarks)):
                if any(word in Transaction.Cleaned_Remarks[i] for word in Not_able_to_do_transcation):
                    Transaction['Sub Categories'][i] =  'Not_able_to_do_transcation'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in wrong_transaction):
                    Transaction['Sub Categories'][i] = 'wrong transaction'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in money_dedected):
                    Transaction['Sub Categories'][i] = 'money dedected'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in transaction_limit):
                    Transaction['Sub Categories'][i] = 'transaction limit'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in payment_failure):
                    Transaction['Sub Categories'][i] = 'payment failure'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in not_refund):
                    Transaction['Sub Categories'][i] = 'not refund'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in error):
                    Transaction['Sub Categories'][i] = 'error'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in other_issue):
                    Transaction['Sub Categories'][i] = 'other issue'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in pending):
                    Transaction['Sub Categories'][i] = 'pending'

                elif any(word in Transaction.Cleaned_Remarks[i] for word in rejection_in_payment):
                    Transaction['Sub Categories'][i] = 'rejection in payment'



            Happy_Customer = df[df.Categories=='Happy Customers']
            Happy_Customer =Happy_Customer.reset_index()
            Happy_Customer = Happy_Customer.drop(['index'],axis =1)

            cashback = ['cashback','offer','reward']
            interface = ['interface']
            security = ['secuity','secure']
            trust_bcz_gov_support = ['indian','govt','government']
            userfriendlyapp = ['easy','user friendly','app','application']


            for i in range(0, len(Happy_Customer.Cleaned_Remarks)):
                if any(word in Happy_Customer.Cleaned_Remarks[i] for word in cashback):
                    Happy_Customer['Sub Categories'][i] =  'Cashback/offer Related'

                elif any(word in Happy_Customer.Cleaned_Remarks[i] for word in interface):
                    Happy_Customer['Sub Categories'][i] = 'Interface'

                elif any(word in Happy_Customer.Cleaned_Remarks[i] for word in security):
                    Happy_Customer['Sub Categories'][i] = 'Security'

                elif any(word in Happy_Customer.Cleaned_Remarks[i] for word in trust_bcz_gov_support):
                    Happy_Customer['Sub Categories'][i] = 'Trust Because of Govt Support'

                elif any(word in Happy_Customer.Cleaned_Remarks[i] for word in userfriendlyapp):
                    Happy_Customer['Sub Categories'][i] = 'User Friendly App'
            Happy_Customer['Sub Categories'] = Happy_Customer['Sub Categories'].replace(np.nan, 'Generic Comment')

            df_ = pd.concat([Transaction, df,Happy_Customer]).drop_duplicates().reset_index(drop=True)
            df_ = df_.sample(frac=1).reset_index(drop=True)
            df_ = df_[['id','product','Remarks','vendor_name','vendor_id','survey_response_id','state','city','rating_response',
               'rating_response_datetime','rating','survey_response','survey_response_datetime','Categories',
               'Sub Categories']]
            df_['Categories'] = df_['Categories'].replace('', np.nan, regex=True)
            df_['Categories'] = df_['Categories'].replace(np.nan, 'Unproductive Comment')
            df_.reset_index(drop=True, inplace=True)
            col = ['Unproductive Comment']
            df_ = df_[~df_['Categories'].isin(col)]
            df =df_

            df['NPS CAT']= ""
            detractor = df[df['rating'] < 7]
            detractor['NPS CAT'] = "Detractor"
            promoter =df[df['rating'] > 8]
            promoter['NPS CAT'] = "Promoter"
            passive = df[(df['rating'] < 9) & (df['rating'] > 6)]
            passive['NPS CAT'] = "passive"

            frames = [detractor,promoter,passive]
            final = pd.concat(frames)

            options = ['Unproductive Comment']
            final_imp_cat = final.loc[~final['Categories'].isin(options)]

            detractor = final_imp_cat[final_imp_cat['rating'] < 7]
            detractor['NPS CAT'] = "Detractor"
            promoter =final_imp_cat[final_imp_cat['rating'] > 8]
            promoter['NPS CAT'] = "Promoter"
            passive = final_imp_cat[(final_imp_cat['rating'] < 9) & (final_imp_cat['rating'] > 6)]
            passive['NPS CAT'] = "Passive"


            frames = [detractor,promoter,passive]
            final_ = pd.concat(frames)

            Promoter_count  = len(final_[final_['NPS CAT']=='Promoter'])
            passive_count   = len(final_[final_['NPS CAT']=='Passive'])
            Detractor_count = len(final_[final_['NPS CAT']=='Detractor'])

            final_['Total Nps Score'] = ''
            final_['Total Nps Score'] = (Promoter_count-Detractor_count)/(Promoter_count+passive_count+Detractor_count)*100

            col = ['Unnamed: 0', 'id', 'product', 'Remarks', 'vendor_name', 'vendor_id',
                  'survey_response_id', 'state', 'city', 'rating_response',
                  'rating_response_datetime', 'rating', 'survey_response',
                  'survey_response_datetime', 'Categories', 'Sub Categories',
                  'NPS CAT', 'Total Nps Score', 'Total Nps Score Cat Wise']
            df = pd.DataFrame(columns=col)

            for i in final_['Categories'].unique():
                j=i
                j = final_[final_['Categories'] == i]
                detractor = j[j['rating'] < 7]
                detractor['NPS CAT'] = "Detractor"
                promoter =j[j['rating'] > 8]
                promoter['NPS CAT'] = "Promoter"
                passive = j[(j['rating'] < 9) & (j['rating'] > 6)]
                passive['NPS CAT'] = "Passive"

                frames = [detractor,promoter,passive]
                a = pd.concat(frames)
                Promoter_count  = len(a[a['NPS CAT']=='Promoter'])
                passive_count   = len(a[a['NPS CAT']=='Passive'])
                Detractor_count = len(a[a['NPS CAT']=='Detractor'])


                a['Total Nps Score Cat Wise'] = ''
                a['Total Nps Score Cat Wise'] = (Promoter_count-Detractor_count)/(Promoter_count+passive_count+Detractor_count)*100
                frames = [df,a]
                df = pd.concat(frames)

            final_df = df[['Categories','rating']]
            final_df['rating']= final_df['rating'].astype(int)
            X = final_df.select_dtypes(include=[object])
            Y = final_df['rating']
            X = X.copy()
            X = pd.get_dummies(X['Categories'])

            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

            model = XGBClassifier()
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)

            def importance_vs_feature_plot(columns, importances):
                df = (pd.DataFrame({'feature' : columns,
                                   'feature importances' : importances})
                    .sort_values('feature importances', ascending=True)
                    .reset_index(drop=True))
                fig, ax = plt.subplots()
                ax.barh(df['feature'], df['feature importances'])

            importance_vs_feature_plot( X_train.columns, model.feature_importances_ )

            output_df = pd.DataFrame({"Categories" : X_train.columns,
                                     "Impact of Categories on NPS" : model.feature_importances_}).sort_values(by="Impact of Categories on NPS", ascending=False)
            df1 = pd.merge(df, output_df, on = 'Categories', how = 'left')

            df2 = df1[df1["Categories"]=='Transaction']
            df2 = df2.replace("",np.nan)
            df2.dropna(subset = ["Sub Categories"], inplace=True)

            X = pd.get_dummies(df2["Sub Categories"]).reset_index(drop=True)
            Y = df2["rating"]
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

            model = XGBClassifier()
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)


            def importance_vs_feature_plot(columns, importances):
                df = (pd.DataFrame({'feature' : columns,
                                   'feature importances' : importances})
                    .sort_values('feature importances', ascending=True)
                    .reset_index(drop=True))
                fig, ax = plt.subplots()
                ax.barh(df['feature'], df['feature importances'])

            importance_vs_feature_plot( X_train.columns, model.feature_importances_ )

            output_df_ = pd.DataFrame({"Sub Categories" : X_train.columns,
                                     "Impact of Sub Categories on NPS" : model.feature_importances_}).sort_values(by="Impact of Sub Categories on NPS", ascending=False)

            df3 = pd.merge(df1, output_df_, on = 'Sub Categories', how = 'left')
            df3 = df3.drop(['Unnamed: 0'],axis= 1)

            from datetime import datetime
            now = datetime.now()
            df3.to_csv(fr'E:\DIvyam\BHIM\quad_day_wise\result{now.strftime("%d_%m_%Y %H_%M_%S")}.csv')
            files = [join(r"E:\DIvyam\BHIM\quad_day_wise", f) for f in listdir(r"E:\DIvyam\BHIM\quad_day_wise")]
            path = files[0]
            url = "https://npci.surveycxm.com:3000/survey/update-quadrant-data"
    #         res = requests.post(url,verify = False, files = {'quadrant_file' :  open(path, 'rb'),'start_date' : '2021-08-20','end_date':'2021-08-20'})
            res = requests.post(url,verify = False, files = {'quadrant_file' :  open(path, 'rb')}, data = {'DB':'photcat','OUT':'csv','SHORT':
                                                                                                          'short','start_date' : start_date,'end_date':end_date})
            print(res.status_code)
            try:
                for f in files:
                    shutil.move(f,'E:\\DIvyam\\BHIM\\data')
            except:
                pass
        dotask(df)
        print(now)
    except Exception as e:
        print(e)

In [2]:
import schedule
import time
schedule.every().day.at("03:15").do(quadrantdaywise)

Every 1 day at 03:15:00 do quadrantdaywise() (last run: [never], next run: 2021-10-02 03:15:00)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(5)

[03:15:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:15:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
200
2021-10-02 03:15:10.521801
[03:15:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:15:08] WARNING: C:/Users/Adminis

In [1]:
import datetime
from datetime import date, timedelta
import requests

now = datetime.datetime.now()
start_date = "{}".format((now-timedelta(hours=24, seconds=0, minutes=0)).strftime("%Y-%m-%d"))
#         start_date = f'2021-09-0{i}'
end_date = start_date
data =  {
    "type": "per_day",
    "survey_id":[85],
    "limit": 1
}
print(start_date)
res = requests.post('https://npci.surveycxm.com:3000/survey/get-merge-xlsx-file',json =data ,verify = False)

url = res.json()['data'][0]['file_url']

2021-10-04


E:\anaconda\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'npci.surveycxm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
import pandas as pd
df = pd.read_excel(url)

In [5]:
res.json()

{'error': False,
 'code': 200,
 'message': 'OK',
 'data': [{'id': 860,
   'type': 'per_day',
   'start_date': '2021-09-30 00:00:00',
   'end_date': '2021-09-30 23:59:59',
   'filename': '2021-09-30000000_2021-09-30235959_1633032000399.xlsx',
   'file_url': 'https://npci.surveycxm.com:3000/free_text_dump_data/2021-09-30000000_2021-09-30235959_1633032000399.xlsx',
   'survey_id': 85,
   'created_at': '2021-09-30T20:00:00.000Z'}]}